# Task4a_Utilization_Prediction

In [159]:
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', None)

## Dataset (Preparation)

In [119]:
# Read in the pre-processed dataset

data = pd.read_pickle("preparedData.pkl")
data

,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,userID,combinedSpaceStationID,timeConnected,idleTime,chargingTime,weekday,month,time,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,194.0,AG-3F30/1-1-179-810,21741.0,5980.0,15761.0,3,1,13:08:54,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,4275.0,AG-1F01/1-1-193-825,32491.0,8416.0,24075.0,3,1,13:36:50,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,344.0,AG-1F03/1-1-193-829,38567.0,29056.0,9511.0,3,1,13:56:35,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,1117.0,AG-1F04/1-1-193-820,9521.0,4794.0,4727.0,3,1,13:59:58,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,334.0,AG-1F06/1-1-193-819,29319.0,13870.0,15449.0,3,1,14:00:01,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56531,2019-07-01 19:13:11+00:00,2019-07-02 00:44:40+00:00,2019-07-01 21:13:56+00:00,6.048,1,2643.0,AG-3F23/1-1-179-797,19889.0,12644.0,7245.0,0,7,19:13:11,275.0,8.25,30.0,374.0,"Mon, 01 Jul 2019 19:13:10 GMT",True,"Tue, 02 Jul 2019 01:27:11 GMT"
56532,2019-07-01 19:43:46+00:00,2019-07-02 02:58:53+00:00,2019-07-02 02:43:53+00:00,41.047,1,394.0,AG-4F38/1-1-191-793,26107.0,900.0,25207.0,0,7,19:43:46,231.0,36.96,160.0,310.0,"Mon, 01 Jul 2019 19:43:58 GMT",True,"Tue, 02 Jul 2019 00:53:46 GMT"
56533,2019-07-01 20:34:30+00:00,2019-07-02 00:30:52+00:00,2019-07-01 21:27:44+00:00,5.390,1,2276.0,AG-3F15/1-1-179-813,14182.0,10988.0,3194.0,0,7,20:34:30,300.0,12.00,40.0,381.0,"Mon, 01 Jul 2019 20:34:29 GMT",True,"Tue, 02 Jul 2019 02:55:30 GMT"
56534,2019-07-01 21:49:12+00:00,2019-07-02 01:37:18+00:00,2019-07-01 23:14:33+00:00,8.399,1,1346.0,AG-3F18/1-1-179-798,13686.0,8565.0,5121.0,0,7,21:49:12,400.0,32.00,80.0,36.0,"Mon, 01 Jul 2019 21:49:37 GMT",True,"Mon, 01 Jul 2019 22:25:12 GMT"


In [120]:
# read in weather data

weather = pd.read_csv("weather_burbank_airport.csv")
weather

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
0,Burbank,2018-01-01 08:53:00,9.0,33.0,Fair,991.75,9.0,0.0,8.0
1,Burbank,2018-01-01 09:53:00,9.0,33.0,Fair,992.08,0.0,0.0,9.0
2,Burbank,2018-01-01 10:53:00,9.0,21.0,Haze,992.08,0.0,0.0,9.0
3,Burbank,2018-01-01 11:53:00,9.0,29.0,Partly Cloudy,992.08,0.0,0.0,9.0
4,Burbank,2018-01-01 12:53:00,8.0,33.0,Fair,992.08,0.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...
29239,Burbank,2021-01-01 03:53:00,13.0,33.0,Fair,986.81,0.0,0.0,13.0
29240,Burbank,2021-01-01 04:53:00,12.0,33.0,Fair,986.81,11.0,0.0,12.0
29241,Burbank,2021-01-01 05:53:00,12.0,33.0,Fair,987.47,9.0,0.0,12.0
29242,Burbank,2021-01-01 06:53:00,11.0,33.0,Fair,987.14,13.0,0.0,11.0


### Combine Data

In [121]:
# drop unnecessary attributes of data df 

data = data.drop(['disconnectTime', 'doneChargingTime', 'kWhDelivered', 'combinedSpaceStationID', 'timeConnected', 'idleTime',
           'chargingTime', 'WhPerMile', 'kWhRequested', 'milesRequested', 'minutesAvailable', 'modifiedAt', 'paymentRequired',
           'requestedDeparture'], axis=1)
data

,connectionTime,siteID,userID,weekday,month,time
0,2020-01-02 13:08:54+00:00,1,194.0,3,1,13:08:54
1,2020-01-02 13:36:50+00:00,1,4275.0,3,1,13:36:50
2,2020-01-02 13:56:35+00:00,1,344.0,3,1,13:56:35
3,2020-01-02 13:59:58+00:00,1,1117.0,3,1,13:59:58
4,2020-01-02 14:00:01+00:00,1,334.0,3,1,14:00:01
...,...,...,...,...,...,...
56531,2019-07-01 19:13:11+00:00,1,2643.0,0,7,19:13:11
56532,2019-07-01 19:43:46+00:00,1,394.0,0,7,19:43:46
56533,2019-07-01 20:34:30+00:00,1,2276.0,0,7,20:34:30
56534,2019-07-01 21:49:12+00:00,1,1346.0,0,7,21:49:12


In [122]:
# drop unnecessary attributes of weather df 

weather = weather.drop(['city', 'cloud_cover', 'cloud_cover_description', 'pressure', 'windspeed', 'felt_temperature'], axis=1)
weather

,timestamp,temperature,precipitation
0,2018-01-01 08:53:00,9.0,0.0
1,2018-01-01 09:53:00,9.0,0.0
2,2018-01-01 10:53:00,9.0,0.0
3,2018-01-01 11:53:00,9.0,0.0
4,2018-01-01 12:53:00,8.0,0.0
...,...,...,...
29239,2021-01-01 03:53:00,13.0,0.0
29240,2021-01-01 04:53:00,12.0,0.0
29241,2021-01-01 05:53:00,12.0,0.0
29242,2021-01-01 06:53:00,11.0,0.0


In [123]:
#convert timestamp attribute into datetime type and drop old timestamp attribute

weather["dateTime"] = pd.to_datetime(weather['timestamp'])
weather = weather.drop(['timestamp'], axis=1)
weather

In [124]:
data['connectionTime'] = data['connectionTime'].dt.tz_localize(None)
data

,connectionTime,siteID,userID,weekday,month,time
0,2020-01-02 13:08:54,1,194.0,3,1,13:08:54
1,2020-01-02 13:36:50,1,4275.0,3,1,13:36:50
2,2020-01-02 13:56:35,1,344.0,3,1,13:56:35
3,2020-01-02 13:59:58,1,1117.0,3,1,13:59:58
4,2020-01-02 14:00:01,1,334.0,3,1,14:00:01
...,...,...,...,...,...,...
56531,2019-07-01 19:13:11,1,2643.0,0,7,19:13:11
56532,2019-07-01 19:43:46,1,394.0,0,7,19:43:46
56533,2019-07-01 20:34:30,1,2276.0,0,7,20:34:30
56534,2019-07-01 21:49:12,1,1346.0,0,7,21:49:12


In [125]:
# drop minutes and seconds of both datetime attributes

data['connectionTime'] = data['connectionTime'].dt.strftime('%Y-%m-%d %H:00:00')
weather['dateTime'] = weather['dateTime'].dt.strftime('%Y-%m-%d %H:00:00')

In [132]:
# drop duplicates in weather to ensure correct merge of both dataframes 

weather.drop_duplicates(subset='dateTime', inplace=True)

In [135]:
# combine both df into a single one along the dateTime and connectionTime attribute

mergedData = pd.merge(data, weather, left_on='connectionTime', right_on='dateTime', how="left")
mergedData

In [158]:
mergedData.isna().sum()

connectionTime        0
siteID                0
userID            15227
weekday               0
month                 0
time                  0
temperature        7461
precipitation      7450
dateTime           7450
dtype: int64

In [156]:
# drop all rows where we don't have entries in the weather attributes

cleanedData = mergedData.dropna(subset=['temperature','precipitation','dateTime'])
cleanedData

,connectionTime,siteID,userID,weekday,month,time,temperature,precipitation,dateTime
0,2020-01-02 13:00:00,1,194.0,3,1,13:08:54,13.0,0.0,2020-01-02 13:00:00
1,2020-01-02 13:00:00,1,4275.0,3,1,13:36:50,13.0,0.0,2020-01-02 13:00:00
2,2020-01-02 13:00:00,1,344.0,3,1,13:56:35,13.0,0.0,2020-01-02 13:00:00
3,2020-01-02 13:00:00,1,1117.0,3,1,13:59:58,13.0,0.0,2020-01-02 13:00:00
4,2020-01-02 14:00:00,1,334.0,3,1,14:00:01,14.0,0.0,2020-01-02 14:00:00
...,...,...,...,...,...,...,...,...,...
56531,2019-07-01 19:00:00,1,2643.0,0,7,19:13:11,29.0,0.0,2019-07-01 19:00:00
56532,2019-07-01 19:00:00,1,394.0,0,7,19:43:46,29.0,0.0,2019-07-01 19:00:00
56533,2019-07-01 20:00:00,1,2276.0,0,7,20:34:30,30.0,0.0,2019-07-01 20:00:00
56534,2019-07-01 21:00:00,1,1346.0,0,7,21:49:12,31.0,0.0,2019-07-01 21:00:00


### Split Data into Sites

In [153]:
privateSiteDf = cleanedData[cleanedData['siteID'] == 1]
privateSiteDf

,connectionTime,siteID,userID,weekday,month,time,temperature,precipitation,dateTime
0,2020-01-02 13:00:00,1,194.0,3,1,13:08:54,13.0,0.0,2020-01-02 13:00:00
1,2020-01-02 13:00:00,1,4275.0,3,1,13:36:50,13.0,0.0,2020-01-02 13:00:00
2,2020-01-02 13:00:00,1,344.0,3,1,13:56:35,13.0,0.0,2020-01-02 13:00:00
3,2020-01-02 13:00:00,1,1117.0,3,1,13:59:58,13.0,0.0,2020-01-02 13:00:00
4,2020-01-02 14:00:00,1,334.0,3,1,14:00:01,14.0,0.0,2020-01-02 14:00:00
...,...,...,...,...,...,...,...,...,...
56531,2019-07-01 19:00:00,1,2643.0,0,7,19:13:11,29.0,0.0,2019-07-01 19:00:00
56532,2019-07-01 19:00:00,1,394.0,0,7,19:43:46,29.0,0.0,2019-07-01 19:00:00
56533,2019-07-01 20:00:00,1,2276.0,0,7,20:34:30,30.0,0.0,2019-07-01 20:00:00
56534,2019-07-01 21:00:00,1,1346.0,0,7,21:49:12,31.0,0.0,2019-07-01 21:00:00


In [154]:
publicSiteDf = cleanedData[cleanedData['siteID'] == 2]
publicSiteDf

,connectionTime,siteID,userID,weekday,month,time,temperature,precipitation,dateTime
16736,2019-01-01 18:00:00,2,558.0,1,1,18:09:17,12.0,0.00,2019-01-01 18:00:00
16737,2019-01-01 19:00:00,2,NaN,1,1,19:18:53,13.0,0.00,2019-01-01 19:00:00
16738,2019-01-01 21:00:00,2,1135.0,1,1,21:05:57,14.0,0.00,2019-01-01 21:00:00
16739,2019-01-01 21:00:00,2,838.0,1,1,21:08:49,14.0,0.00,2019-01-01 21:00:00
16740,2019-01-01 21:00:00,2,NaN,1,1,21:10:03,14.0,0.00,2019-01-01 21:00:00
...,...,...,...,...,...,...,...,...,...
44883,2020-12-29 00:00:00,2,1746.0,1,12,00:22:25,8.0,0.25,2020-12-29 00:00:00
44884,2020-12-29 22:00:00,2,1082.0,1,12,22:21:20,16.0,0.00,2020-12-29 22:00:00
44885,2020-12-30 19:00:00,2,1746.0,2,12,19:32:38,17.0,0.00,2020-12-30 19:00:00
44886,2020-12-30 23:00:00,2,1082.0,2,12,23:02:11,18.0,0.00,2020-12-30 23:00:00


## Model for private Site

## Model for public Site